In [1]:
!pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!sudo apt-get install espeak

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement espeak (from versions: none)
ERROR: No matching distribution found for espeak


In [1]:
import numpy as np
import pandas as pd
import os
import pyttsx3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import  pickle
import warnings
warnings.filterwarnings("ignore", message="X has feature names")

In [2]:
with open('fraud_detection_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [3]:
with open('feature_engineer_df.pkl', 'rb') as f:
    df= pickle.load(f)

In [22]:
df.head(10)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
1533542,154,2,259500.87,2602539.43,2862040.30,2277849.22,2018348.35,0
6185610,567,1,26757.98,108635.83,81877.85,0.00,0.00,0
3005197,232,1,11223.25,40997.65,29774.40,0.00,0.00,0
822808,40,3,284852.00,0.00,0.00,1526139.91,1810991.91,0
671317,36,1,27152.63,0.00,0.00,0.00,0.00,0
1173345,132,2,609102.43,11601.00,620703.43,22711.00,0.00,0
3097574,235,0,196144.10,509036.34,312892.24,1033290.92,1229435.02,0
946939,44,0,196712.81,0.00,0.00,1024665.07,1221377.88,0
2664472,210,0,317420.65,0.00,0.00,3013526.82,3330947.47,0
3442149,257,0,78910.43,0.00,0.00,537847.37,616757.80,0


In [4]:
for dirname, _, filenames in os.walk('PS_20174392719_1491204439457_log.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
dfd = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [ ]:
def creat_transaction_description(row):
    return(f"At step {row['step']}, a {row['type']} of ${row['amount']} was made "
           f"from {row['nameOrig']}(Old Balance: $ {row['oldbalanceOrg']}, New balance: $ {row['newbalanceOrig']})"
 f"to {row['nameDest']} (Old Balance: $ {row['oldbalanceDest']}, New balance: $ {row['newbalanceDest']}) .")

# Apply the function to create a new 'description' column
dfd['description'] = dfd.apply(creat_transaction_description, axis=1)  

# Display the first few descriptions
print(dfd[['description']].head(5))   


In [ ]:
# Initialize Lemmatizer & Stop Words
lemmatizer = WordNetLemmatizer()  # Create a lemmatizer Object
stop_words = set(stopwords.words('english'))   # Get the List of Stop Words

# Function to Preprocess 
def preprocess_text(text):
     # Step 1: Tokenize
    tokens = word_tokenize(text.lower()) 

    # Step 2: Lemmatzie and remove stop words
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token not in stop_words]  

    # Step 3: Join tokens back into a single string
    return ' '.join(tokens)  


# Apply preprocessing to the 'description' column
df['cleaned_description'] = df['description'].apply(preprocess_text)
print(df[['cleaned_description']].head(30))

In [ ]:
# Step 4: Feature Extraction 

In [ ]:
num_iterations = 16
sample_size = 5000

# Initialize a list to store each sample
samples_list = []

# Loop through 16 times to get different random samples
for i in range(num_iterations):
    
    # Randomly sample 5000 rows in each iteration
    df_sample = df.sample(n=sample_size, random_state=i)
    df_sample = df_sample.reset_index(drop=True)
    
    # Store the sampled data
    samples_list.append(df_sample)
    
    # Display the shape of each sample
    print(f"Sample {i+1} shape: {df_sample.shape}")

# After this, 'samples_list' contains 16 separate DataFrames each with 5000 randomly selected rows.

from sklearn.feature_extraction.text import TfidfVectorizer

#Intialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=1000)   

# Loop through each sample in the list and apply TF-IDF
for i, sample in enumerate(samples_list):
    
    # Ensure that 'cleaned_description' column exists in the sample
    if 'cleaned_description' in sample.columns:
        
        # Fit and transform the 'cleaned_description' column using TF-IDF
        X_tfidf = vectorizer.fit_transform(sample['cleaned_description'])
        
        # Convert the sparse matrix to a dense array
        X_tfidf_array = X_tfidf.toarray()
        
        # Display the shape of the TF-IDF matrix for each sample
        print(f"Sample {i+1} TF-IDF shape: {X_tfidf_array.shape}")
    else:
        print(f"Sample {i+1} does not contain 'cleaned_description' column.")  

In [ ]:
def text_to_speech(text):
    engine = pyttsx3.init()   # Initialize the pyttsx3
    engine.say(text)
    engine.runAndWait()   # Ensure the speech is processed before moving on to the next iteration
    engine.setProperty('rate', 150)  # Speed of speech
    engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)

# Use 'samples_list' which is a list of DataFrames, each containing the 'cleaned_description' and numerical features.
for i, df_sample in enumerate(samples_list):
    
    #Select Numerical features
    numerical_features = df_sample[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest' ]].values
    
    #Combine TF-IDF features with numerical features
    X_combined = np.hstack((X_tfidf.toarray(), numerical_features))
    
    # Define the target variable: "isFraud"
    y = df_sample['isFraud']
    
    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size = 0.2, random_state = 42)

    #Train a Random Forest model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Evaluation
    accuracy = model.score(X_test, y_test)
    print(f"Sample {i+1} Model Accuracy: {accuracy}")
    
    # Print and convert accuracy to speech
    text_to_speech(f"Sample {i+1} Its Model Accuracy is {accuracy}")